## a) Find the posterior for $\tilde P$. Compute the expectation 
*The transition matrix  $\tilde P$ of the embedded chain is unknown, and we will use data from Figure 1 to learn about it. Let $\tilde P_{1}, \tilde P_{2}, \tilde P_{4}$, and $\tilde P_{4}$ be the four rows of $\tilde P$. \
Find the posterior given the data in the figure: Express the posterior using parametric distributions with explicit parameters. Compute the expected value of this posterior distribution.*

For this task we just look at the embedded discrete chain.
We are given 20 observations $x_{0}, \dots, x_{20}$ in the state space $\{ 1,2,3,4 \}$. 
The likelyhood can be formulated as: 

$$
\pi(x_{0}, \dots, x_{20} | \tilde P) = \pi(x_{0}) \prod_{r=1}^{20} \pi(x_{r} | x_{r-1}, \tilde P) = \pi(x_{0}) \prod_{r=1}^{20} \tilde P_{x_{r-1}, x_{r}} = \pi(x_{0}) \prod_{i=k}^4 \prod_{j=1}^4 (P_{ij})^{c_{ij}}
$$

The following values $c_{ij}$ give the count of transition between 2 states $i,j$:

| $c_{ij}$ | **1** | **2** | **3** | **4** |
| -------- | ----- | ----- | ----- | ----- |
| **1**    | 0     | 2     | 0     | 5     |
| **2**    | 4     | 0     | 1     | 0     |
| **3**    | 0     | 0     | 0     | 1     |
| **4**    | 2     | 3     | 0     | 0     |

For the rows of $\tilde P$, we are given the priors: 
$$
\begin{array}{l l l}{{\displaystyle\tilde{P}_{1}}}&{{\sim}}&{{\mathrm{Dirichlet}(0,1,1,1)}}\\ {{\tilde{P}_{2}}}&{{\sim}}&{{\mathrm{Dirichlet}(1,0,1,1)}}\\ {{\displaystyle\tilde{P}_{3}}}&{{\sim}}&{{\mathrm{Dirichlet}(1,1,0,1)}}\\ {{\displaystyle\tilde{P}_{4}}}&{{\sim}}&{{\mathrm{Dirichlet}(1,1,1,0)}}\end{array}
$$
So the Prior for $\tilde P$ can be formulated as: 
$$
\pi(\tilde P) = \prod_{i=1}^4 \mathrm{Dirichlet}(\tilde P_{i}; \alpha_{i}) \propto \prod_{i=1}^4 \prod_{j=1}^4 (P_{ij})^{\alpha_{ij} - 1}
$$
So for the posterior we see: 

$$
\begin{align}
\pi(\tilde P | x_{0}, \dots x_{20}) &\propto \pi(x_{0}, \dots, x_{20} | \tilde P) \pi(\tilde P) \\
&\propto \prod_{i=k}^4 \prod_{j=1}^4 (P_{ij})^{c_{ij}} \prod_{i=1}^4 \prod_{j=1}^4 (P_{ij})^{\alpha_{ij} - 1} \\
&\propto \prod_{i=1}^4 \prod_{j=1}^4 (P_{ij})^{c_{ij} + \alpha_{ij} - 1} 
\propto \prod_{i=1}^4 \mathrm{Dirichlet}( \tilde P_{i}; \alpha_{i} + c_{i})
\end{align}
$$
Thus the posterior is given by the distribution 
$$
\pi(\tilde P | x_{0}, \dots x_{20}) = \mathrm{Dirichlet}( \tilde P_{1}; 0, 3, 1, 6) \mathrm{Dirichlet}( \tilde P_{2}; 5, 0, 2, 1) \mathrm{Dirichlet}( \tilde P_{3}; 1,1,0,2) \mathrm{Dirichlet}( \tilde P_{4}; 3,4,1,0)
$$
The expectation for one entry of a Dirichlet Distribution is given by: 
$$
E[\tilde P_{ij}] = E[\mathrm{Dirichlet}( \tilde P_{i}; \alpha_{i})_{j}] = \frac{a_{ij}}{\sum_{k=1}^4 \alpha_{ik}}
$$

In [21]:
time_s1 <- c(1, 1, 1, 3.4, 1.5, 1.2, 2.5)
sum_time_s1 <- sum(time_s1)
len_time_s1 <- length(time_s1)

time_s2 <- c(1, 1.1, 1.8, 1, 1.8)
sum_time_s2 <- sum(time_s2)
len_time_s2 <- length(time_s2)

time_s3 <- c(1.1)
sum_time_s3 <- sum(time_s3)
len_time_s3 <- length(time_s3)

time_s4 <- c(1.4, 1, 1.8, 1.5, 1.7, 2.2)
sum_time_s4 <- sum(time_s4)
len_time_s4 <- length(time_s4)

print(sum_time_s1 + sum_time_s2 + sum_time_s3 + sum_time_s4)

[1] 29


In [20]:
time_seq=c(0,1,2.4,3.4,4.4,5.4,6.4,7.5,10.9,12.7,14.5,16,17.5,18.5,19.6,21.3,22.5,24.3,26.8,29)
state_seq=c(1,4,1,4,2,1,2,1,4,2,1,4,2,3,4,1,2,1,4)

holding_times = list(c(), c(), c(), c())
for(i in seq_along(state_seq)) {
  append(holding_times[state_seq[i]], time_seq[i+1] -  time_seq[i])
}
holding_times

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

In [7]:
# alpha parameters for the prior distributions
alpha_prior <- matrix(
  c(0, 1, 1, 1,   # alpha1
    1, 0, 1, 1,   # alpha2
    1, 1, 0, 1,   # alpha3
    1, 1, 1, 0),  # alpha4
  nrow = 4, byrow = TRUE
)

transition_counts <- matrix(
  c(0, 2, 0, 5,   # alpha1
    4, 0, 1, 0,   # alpha2
    0, 0, 0, 1,   # alpha3
    2, 3, 0, 0),  # alpha4
  nrow = 4, byrow = TRUE
)

# Calculate the alpha parameters for the posterior
alpha_matrix <- alpha_prior + transition_counts

# Function to compute Dirichlet expectation
# returns a vector
dirichlet_expectation <- function(alpha_row) {
  total_alpha <- sum(alpha_row)
  return(alpha_row / total_alpha)
}

# Calculate the expectations for all rows
expectations <- t(apply(alpha_matrix, 1, dirichlet_expectation))

# Print the expectations
print(expectations)


      [,1] [,2]  [,3]  [,4]
[1,] 0.000 0.30 0.100 0.600
[2,] 0.625 0.00 0.250 0.125
[3,] 0.250 0.25 0.000 0.500
[4,] 0.375 0.50 0.125 0.000



## b) Obtain a posterior distribution for q. Compute the expected value. 
*Now assume that the process is a homogeneous Markov chain. Let $q = (q_{1}, q_{2}, q_{3},q_{4})$ be the vector of parameters for the Exponential distributions of the holding times at the four different states, respectively. Assume independent priors for each qi proportional to $1/qi$. 
Now use the data from the figure to obtain a posterior distribution for q. 
Express the posterior using parametric distributions with explicit parameters. 
Compute the expected value of the posterior distribution.*

The parameters for the exponential distribution of the holding times are given by $q = (q_{1}, q_{2}, q_{3},q_{4})$. Let the observed holding times for a state be $x_{1}, \dots x_{n}$, the values are the following:

|     | $x_1$ | $x_{2}$ | $x_{3}$ | $x_{4}$ | $x_{5}$ | $x_{6}$ | $x_{7}$ |
| --- | ----- | ------- | ------- | ------- | ------- | ------- | ------- |
| 1   | 1     | 1       | 1       | 3.4     | 1.5     | 1.2     | 2.5     |
| 2   | 1     | 1.1     | 1.8     | 1       | 1.8     | -       | -       |
| 3   | 1.1   |         |         |         |         |         |         |
| 4   | 1.4   | 1       | 1.8     | 1.5     | 1.7     | 2.2     | -       |

The likelyhood for each state indepentently is thus given by the exponential distribution: 
$$
\pi(x_{1}, \dots x_{n} | q_{i}) = \prod_{i=1}^n \mathrm{Exponential}(x_{j} ; q_{i}) = q_{i}^n \exp\left( -q_{i} \sum_{j=1}^n x_{j} \right)
$$
The priors are given by independent $\pi(q_{i}) = \frac{1}{q_{i}}$. 
For the posteriors we thus get indepentently: 
$$
\pi(q_{i} | x_{1}, \dots x_{n}) \propto \pi(x_{1}, \dots x_{n} | q_{i}) \pi(q_{i}) = q_{i}^{n-1} \exp\left( -q_{i} \sum_{j=1}^n x_{j} \right) \propto \mathrm{Gamma}(q_{i}, n, \sum_{j=1}^n x_{j} )
$$
$$
\begin{align}
q_1 &\sim \text{Gamma}(7, 11.6) \\
q_2 &\sim \text{Gamma}(5, 6.7) \\  
q_3 &\sim \text{Gamma}(1, 1.1) \\ 
q_4 &\sim \text{Gamma}(6, 9.6) \\
\end{align}
$$
The expectation of a Gamma Distribution is: $E[X] = \frac{\alpha}{\beta}$, so we get the expected rates:
1

In [23]:
number_visited <- c(7, 5, 1, 6)
holding_times <- c(11.6, 6.7, 1.1, 9.6)

number_visited / holding_times

[1] 0.6034483 0.7462687 0.9090909 0.6250000

## c) Write R code which takes as input the transition matrix ˜ P of a continuous time
discrete state-space Markov chain and a vector q of parameters for the
distributions of the holding times of the chain, and outputs the long-term
probability that the process will be in state 3.
Apply the code to the expected values found in questions (a) and (b) and
report the result.